# Real World Embeddings: Diet & Food 🥗

Welcome to the **Real World**! 
In the previous lesson, we used a "Toy Model" with 3 simple dimensions (Fruit, Dog, Cat). 
Now, we will use a powerful, pre-trained AI model to understand complex sentences about food and diet.

**What we will do:**
1.  **Setup**: Prepare our tools and environment.
2.  **Embed**: Turn 15 diet-related sentences into "Vectors" (lists of numbers).
3.  **Store**: Save these vectors into **Qdrant** (our Vector Database).
4.  **Search**: Ask a question like "I want to build muscle" and find the most relevant sentences.

## 1. Setup & Environment

### Why do we need a Virtual Environment?
It is best practice to keep your project dependencies isolated. This prevents conflicts between different projects (e.g., Project A needs `numpy` 1.0, Project B needs `numpy` 2.0).

Run the cell below to create a virtual environment `.venv`. 
**Note:** After running this, if you are in VS Code, you should select this new `.venv` as your Kernel/Interpreter.

In [1]:
# Create a virtual environment named .venv
!python3 -m venv .venv

# Note: To use this, you must select '.venv' as your Kernel in VS Code (Top Right Corner)

### Install Libraries
We need two main libraries:
1.  `sentence-transformers`: The AI library that turns text into vectors.
2.  `qdrant-client`: The tool that lets us talk to the Qdrant database.

We also install `pandas` to help us visualize data nicely.

In [2]:
%pip install sentence-transformers qdrant-client pandas

  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached grpcio-1.76.0-cp313-cp313-macosx_11_0_universal2.whl.metadata (3.7 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 34.5 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 30.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 26.6 MB/s  0:00:00eta 0:00:01
Using cached grpcio-1.76.0-cp313-cp313-macosx_11_0_universal2.whl (11.8 MB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 16.4 MB/s  0:00

## 2. Load the Embedding Model 🧠

We are using a model called `all-MiniLM-L6-v2`.
*   **MiniLM**: It's a smaller, faster version of the famous BERT model.
*   **L6**: It has 6 layers of "neurons".
*   **v2**: Version 2.

This model has read billions of sentences. It knows that "King" is related to "Queen", and "Protein" is related to "Muscle".
It converts any text into a list of **384 numbers**.

In [3]:
from sentence_transformers import SentenceTransformer

# Download and load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/raghunandana.sanur/Desktop/test-repo/vector-database-demo-2026/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. The Data (Our Knowledge Base) 📚

Here represent 15 diverse sentences about food. 
We have Keto, Vegan, Paleo, Junk Food, and Healthy Eating examples.
This list is what we will search through later.

In [6]:
sentences = [
    "I follow a strict ketogenic diet with high fat and low carbs.",
    "Veganism is a lifestyle that excludes all animal products.",
    "I love eating pepperoni pizza implies a lot of cheese.",
    "Paleo diet focuses on whole foods like meat and vegetables.",
    "My daily calories come from sugary soda and fast food.",
    "A plant-based diet is rich in fiber and vitamins.",
    "I am allergic to gluten so I avoid bread and pasta.",
    "Intermittent fasting helps me manage my weight.",
    "Protein shakes are great for muscle recovery after a workout.",
    "I eat a balanced diet with plenty of fruits and water.",
    "Fried chicken and french fries are my favorite cheat meal.",
    "Mediterranean diet involves olive oil and fresh fish.",
    "Sugar-free diets can help reduce inflammation.",
    "Carnivore diet means eating only meat and animal products.",
    "I count my macros to ensure I get enough protein."
]

## 4. Transformation (Text -> Numbers) 🔢

We use `model.encode()` to pass our text through the Neural Network.
The output `embeddings` is a list of vectors.

In [10]:
embeddings = model.encode(sentences)

print(f"Created {len(embeddings)} vectors.")
print(f"{embeddings}")
print(f"Shape of one vector: {embeddings[0].shape}")
# 384 dimensions means each sentence is described by 384 features!

Created 15 vectors.
[[ 0.03987455  0.01822555  0.0602175  ... -0.03542068  0.02106852
   0.02349543]
 [ 0.012781    0.0123942   0.01697932 ... -0.08436691  0.08080513
   0.02343985]
 [-0.02017428 -0.01320761  0.03650714 ...  0.07151126  0.03967074
  -0.076828  ]
 ...
 [-0.00504594  0.0077768   0.04835589 ... -0.09266385  0.02395846
  -0.01080788]
 [ 0.00324737  0.08738441  0.02378584 ...  0.0184341   0.06495902
   0.04318139]
 [-0.00763889 -0.06681007 -0.00521442 ... -0.05119191 -0.06289448
  -0.09797092]]
Shape of one vector: (384,)


## 5. Store in Qdrant 💾

Now we have the vectors, but they are just in memory. Let's save them to a Database.

1.  **Connect**: We connect to Qdrant running on `localhost:6333`.
2.  **Create Collection**: We make a bucket named `diet_facts`. We MUST tell Qdrant the vector size (**384**).
3.  **Upsert**: We upload our data. Each item needs:
    *   **ID**: A unique number (0, 1, 2...)
    *   **Vector**: The 384 numbers we just generated.
    *   **Payload**: The original text (so we can read it back).

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# 1. Connect
client = QdrantClient(url="http://localhost:6333")

collection_name = "diet_facts"

# 2. Create Collection (Refresh if exists)
if client.collection_exists(collection_name):
    client.delete_collection(collection_name)

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)
print(f"Collection '{collection_name}' created!")

# 3. Prepare Data for Upsert
points = []
for idx, (sentence, vector) in enumerate(zip(sentences, embeddings)):
    points.append(PointStruct(
        id=idx,
        vector=vector.tolist(),
        payload={"text": sentence}
    ))

# 4. Upload
client.upsert(
    collection_name=collection_name,
    points=points
)
print(f"Successfully uploaded {len(points)} facts to Qdrant.")

## 6. Semantic Search 🔎

This is the magic part.
We want to find sentences related to **"Muscle"** and **"Healthy Eating"**.

**The Process:**
1.  **Query**: Write a new sentence (Query).
2.  **Encode**: Convert that Query into a Vector (using the SAME model).
3.  **Search**: Send that Vector to Qdrant. Qdrant finds the closest vectors in the database.
4.  **Result**: We get back the most similar facts!

In [ ]:
query_text = "I want to build muscle and eat healthy"
print(f"Query: '{query_text}'")

# 1. Encode the Query
query_vector = model.encode(query_text)

# 2. Search in Qdrant
search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=3  # Give me the top 3 best matches
).points

# 3. Print Results
print("\n--- Top 3 Results ---")
for hit in search_results:
    print(f"Score: {hit.score:.4f} | Text: {hit.payload['text']}")

### Why did it pick these?
You should see sentences about **Protein**, **Macros**, and **Plant-based** diets.
*   Our query mentioned "Muscle". The model knows "Protein" is related to Muscle.
*   Our query mentioned "Healthy". The model knows "Plant-based" and "Macros" are related to health.
*   It likely **IGNORED** the "Pizza" and "Fries" sentences because they are semantically far away from "Healthy".